<a href="https://colab.research.google.com/github/somyayadav14/Email-Classification-System/blob/main/Email%20classification%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
import joblib
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [110]:
nlp = spacy.load("en_core_web_sm")

PII_PATTERNS = {
    "email": r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+",
    "phone_number": r"\b\d{10}\b",
    "dob": r"\b\d{2}/\d{2}/\d{4}\b",
    "aadhar_num": r"\b\d{4}\s\d{4}\s\d{4}\b",
    "credit_debit_no": r"\b(?:\d[ -]*?){13,16}\b",
    "cvv_no": r"\b\d{3}\b",
    "expiry_no": r"\b(0[1-9]|1[0-2])\/\d{2}\b"
}

def mask_pii(text):
    original_entities = []
    masked_text = text
    offset = 0

    # Regex masking
    for entity, pattern in PII_PATTERNS.items():
        for match in re.finditer(pattern, masked_text):
            start, end = match.span()
            original = match.group()
            mask = f"[{entity}]"

            original_entities.append({
                "position": [start, start + len(mask)],
                "classification": entity,
                "entity": original
            })

            masked_text = masked_text[:start] + mask + masked_text[end:]

    # NER-based name masking
    doc = nlp(masked_text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            start, end = ent.start_char, ent.end_char
            original = ent.text
            masked_text = masked_text[:start] + "[full_name]" + masked_text[end:]
            original_entities.append({
                "position": [start, start + len("[full_name]")],
                "classification": "full_name",
                "entity": original
            })

    return masked_text, original_entities



In [111]:
def train_model(df):
    df["text"] = df["subject"] + " " + df["body"]
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    model.fit(df["text"], df["label"])
    joblib.dump(model, "models/email_model.joblib")

In [112]:
df=pd.read_csv('/content/combined_emails_with_natural_pii.csv')
df

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request
...,...,...
23995,Subject: Problem mit der HP DeskJet 3755 WLAN-...,Problem
23996,Subject: Problemas com a impressora HP DeskJet...,Incident
23997,Subject: Problema urgente con el envío\n\nEsti...,Incident
23998,"Subject: \n\nCher Service Client, nous rencont...",Incident


In [113]:
# View the structure
print(df.head())
print(df.info())

                                               email      type
0  Subject: Unvorhergesehener Absturz der Datenan...  Incident
1  Subject: Customer Support Inquiry\n\nSeeking i...   Request
2  Subject: Data Analytics for Investment\n\nI am...   Request
3  Subject: Krankenhaus-Dienstleistung-Problem\n\...  Incident
4  Subject: Security\n\nDear Customer Support, I ...   Request
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   24000 non-null  object
 1   type    24000 non-null  object
dtypes: object(2)
memory usage: 375.1+ KB
None


In [114]:
# Select numeric columns for plotting
numeric_df = df.select_dtypes(include=['number'])


In [116]:
import re

pii_store = {}

PII_PATTERNS = {
    "full_name": r"\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)+)\b",
    "email": r'\b[\w\.-]+@[\w\.-]+\.\w+\b',
    "phone_number": r'\b\d{10}\b',
    "dob": r'\b(?:\d{1,2}[-/]){2}\d{2,4}\b',
    "aadhar_num": r'\b\d{4}[- ]\d{4}[- ]\d{4}\b',
    "credit_debit_no": r'\b(?:\d{4}[- ]?){3}\d{4}\b',
    "cvv_no": r'\b\d{3}\b',
    "expiry_no": r'\b(0[1-9]|1[0-2])/?([0-9]{2,4})\b'
}

def mask_pii(text):
    masked_text = text
    for entity, pattern in PII_PATTERNS.items():
        matches = re.findall(pattern, masked_text)
        count = 0
        for match in matches:
            if isinstance(match, tuple):  # if regex captures groups
                match = " ".join(match)
            tag = f"[{entity}_{count}]"
            pii_store[tag] = match
            masked_text = masked_text.replace(match, tag, 1)
            count += 1
    return masked_text

def restore_pii(text):
    for tag, value in pii_store.items():
        text = text.replace(tag, value)
    return text



In [117]:
pip install scikit-learn pandas

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import pickle


In [102]:
# Load dataset
df = pd.read_csv("/content/combined_emails_with_natural_pii.csv")
df

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request
...,...,...
23995,Subject: Problem mit der HP DeskJet 3755 WLAN-...,Problem
23996,Subject: Problemas com a impressora HP DeskJet...,Incident
23997,Subject: Problema urgente con el envío\n\nEsti...,Incident
23998,"Subject: \n\nCher Service Client, nous rencont...",Incident


In [14]:
# Check for the correct column name
print(df.columns)  # Print all column names to inspect

Index(['email', 'type'], dtype='object')


In [15]:
# Assuming the correct column name is 'label' (replace with actual name if different)
X = df['email']
y = df['type']

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [77]:
# Model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train_vec, y_train)

RandomForestClassifier()

In [19]:
# Evaluation
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Change       0.95      0.62      0.75       479
    Incident       0.65      0.98      0.78      1920
     Problem       0.83      0.11      0.19      1009
     Request       0.87      0.92      0.90      1392

    accuracy                           0.74      4800
   macro avg       0.83      0.66      0.65      4800
weighted avg       0.78      0.74      0.69      4800



In [20]:
import os

# Create the 'model' directory if it doesn't exist
os.makedirs("model", exist_ok=True)

# Save
with open("model/vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
with open("model/classifier.pkl", "wb") as f:
    pickle.dump(model, f)

#2. Deep Learning (LSTM via Keras)

In [21]:
!pip install tensorflow keras pandas scikit-learn

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [24]:
# Load data
df = pd.read_csv("/content/combined_emails_with_natural_pii.csv")
texts = df['email'].tolist()
labels = df['type'].tolist()

In [25]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [26]:
# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, padding='post', maxlen=100)


In [27]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2)


In [28]:
# Model
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(len(set(labels)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


600/600 ━━━━━━━━━━━━━━━━━━━━ 39s 59ms/step - accuracy: 0.4190 - loss: 1.2636 - val_accuracy: 0.5879 - val_loss: 1.0323
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 41s 59ms/step - accuracy: 0.5001 - loss: 1.1513 - val_accuracy: 0.5279 - val_loss: 1.1202
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 40s 57ms/step - accuracy: 0.5268 - loss: 1.1236 - val_accuracy: 0.5527 - val_loss: 1.0844
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 43s 61ms/step - accuracy: 0.5559 - loss: 1.0560 - val_accuracy: 0.6650 - val_loss: 0.7597
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.6550 - loss: 0.7284 - val_accuracy: 0.6815 - val_loss: 0.6997


In [29]:
# Save model
model.save("lstm_model.h5")

#3.LLM-Based Classification (BERT with Hugging Face)

In [30]:
!pip install transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [31]:
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle

In [78]:
# Load and encode
# Replace 'data/masked_emails.csv' with the correct path to your file
df = pd.read_csv("/content/combined_emails_with_natural_pii.csv")
texts = df['email'].tolist()  # Replace 'masked_body' if it's a different column
labels = LabelEncoder().fit_transform(df['type'])  # Replace 'category' if it's a different column
df

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request
...,...,...
23995,Subject: Problem mit der HP DeskJet 3755 WLAN-...,Problem
23996,Subject: Problemas com a impressora HP DeskJet...,Incident
23997,Subject: Problema urgente con el envío\n\nEsti...,Incident
23998,"Subject: \n\nCher Service Client, nous rencont...",Incident


In [79]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


In [80]:
# Hugging Face Dataset
dataset = Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'label': labels
})

In [82]:
# Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(labels)))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
import wandb
# ... (your other imports and code) ...

# Login to wandb
wandb.login()


# ... (rest of your code) ...

True

In [39]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_output",
    # Replace 'evaluation_strategy' with 'eval_steps' or 'evaluation_strategy="steps"' if necessary
    eval_steps = 100, #  evaluate every 100 steps
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)



# Save model
model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_model")

('bert_model/tokenizer_config.json',
 'bert_model/special_tokens_map.json',
 'bert_model/vocab.txt',
 'bert_model/added_tokens.json')

save the Model

In [40]:
model.save_pretrained("models/email_classifier")
tokenizer.save_pretrained("models/email_classifier")


('models/email_classifier/tokenizer_config.json',
 'models/email_classifier/special_tokens_map.json',
 'models/email_classifier/vocab.txt',
 'models/email_classifier/added_tokens.json')

#3.System Integration Pipeline Breakdown

Email Receiving

In [41]:
# utils.py

def receive_email(subject: str, body: str) -> dict:
    return {"subject": subject, "body": body}


Masking Email Content

In [42]:
# utils.py
def receive_email(subject: str, body: str) -> dict:
    return {"subject": subject, "body": body}

import re

def mask_email_content(text: str) -> str:
    # Mask emails
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL_MASKED]', text)
    # Add more PII masking as needed
    return text

def mask_email(email: dict) -> dict:
    return {
        "subject": mask_email_content(email["subject"]),
        "body": mask_email_content(email["body"])
    }


Classification Using Fine-Tuned Model



In [43]:
# models.py
from transformers import BertTokenizer, BertForSequenceClassification
import torch


In [44]:
# Load model and tokenizer (you can load from local or Hugging Face Hub)
tokenizer = BertTokenizer.from_pretrained("models/email_classifier")
model = BertForSequenceClassification.from_pretrained("models/email_classifier")
model.eval()

label_map = {0: "Billing", 1: "Technical Issue", 2: "Account Problem"}


In [45]:
def classify_email(subject: str, body: str) -> dict:
    input_text = subject + " " + body
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
        confidence = torch.softmax(outputs.logits, dim=1).max().item()

    return {
        "label": label_map[predicted_class],
        "confidence": round(confidence, 3)
    }

#4. API Development & Deployment:


In [46]:
!pip install fastapi uvicorn pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00


In [84]:
#api.py
from fastapi import FastAPI
from pydantic import BaseModel





In [85]:
app = FastAPI(title="Email Classifier API", description="Classify emails after masking PII", version="1.0")


In [86]:
class EmailRequest(BaseModel):
    subject: str
    body: str

@app.get("/")
def home():
    return {"message": "Welcome to the Email Classifier API 🚀"}

@app.post("/classify")
def classify_email(email: EmailRequest):
    result = run_pipeline(email.subject, email.body)
    return {
        "masked_subject": email.subject,
        "masked_body": email.body,
        "classification": result["label"],
        "confidence": result["confidence"]
    }

In [91]:
from pydantic import BaseModel # Importing BaseModel from pydantic

class EmailRequest(BaseModel):
    subject: str
    body: str

@app.get("/")
def home():
    return {"message": "Welcome to the Email Classifier API 🚀"}

@app.post("/classify")
def classify_email(email: EmailRequest):
    result = run_pipeline(email.subject, email.body)
    return {
        "masked_subject": email.subject,
        "masked_body": email.body,
        "classification": result["label"],
        "confidence": result["confidence"]
    }

Post(http://localhost:8000/classify)

In [92]:
{
  "subject": "Login Issue",
  "body": "I can't log in to my account. My email is user@example.com"
}


{'subject': 'Login Issue',
 'body': "I can't log in to my account. My email is user@example.com"}

In [93]:
{
  "masked_subject": "Login Issue",
  "masked_body": "I can't log in to my account. My email is [EMAIL_MASKED]",
  "classification": "Account Problem",
  "confidence": 0.942
}


{'masked_subject': 'Login Issue',
 'masked_body': "I can't log in to my account. My email is [EMAIL_MASKED]",
 'classification': 'Account Problem',
 'confidence': 0.942}

In [121]:
# Train this locally, just once
import pandas as pd
from models import train_model


ImportError: cannot import name 'train_model' from 'models' (unknown location)

In [95]:
# models.py
import pandas as pd
# Placeholders for dependencies that should be imported explicitly
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

def train_model():
    """
    This is a placeholder for your model training logic.
    Replace this with your actual model training code.
    """
    print("Model training logic goes here")
    # Add your training code here, e.g., loading data, preprocessing, training a model, etc.
    # Example:
    # df = pd.read_csv("your_data.csv")
    # ... (your model training code) ...
    pass  # Remove this line when you add the actual training code

In [101]:
df = pd.read_csv("/content/combined_emails_with_natural_pii.csv")
df

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request
...,...,...
23995,Subject: Problem mit der HP DeskJet 3755 WLAN-...,Problem
23996,Subject: Problemas com a impressora HP DeskJet...,Incident
23997,Subject: Problema urgente con el envío\n\nEsti...,Incident
23998,"Subject: \n\nCher Service Client, nous rencont...",Incident
